### WebdriverIO Langchain tools

### Load local LLM 

In [1]:
# !pip install -U langchain-openai
# !pip install dotenv

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.5,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [3]:
# from langchain_ollama import ChatOllama

# llm = ChatOllama(
#     base_url="http://localhost:11434",
#     model="deepseek-r1:8b",
#     temperature=0.5,)

### open_browser tool definition 

In [4]:
from langchain.tools import tool
import requests

SERVER_URL = "http://localhost:3000"

@tool
def open_browser(_=None) -> str:
    """Open Chrome browser using WebdriverIO server."""
    try:
        resp = requests.post(f"{SERVER_URL}/open")
        return resp.text
    except Exception as e:
        return f"Error calling WebdriverIO server: {e}"

@tool
def navigate_to_url(url: str) -> str:
    """Navigate to the given URL using WebdriverIO server."""
    try:
        resp = requests.post(f"{SERVER_URL}/navigate", json={"url": url})
        return resp.text
    except Exception as e:
        return f"Error calling WebdriverIO server: {e}"

@tool
def click_action(element_selector: str) -> str:
    """Click an element in Chrome browser using WebdriverIO server."""
    try:
        resp = requests.post(f"{SERVER_URL}/click", json={"selector": element_selector})
        return resp.text
    except Exception as e:
        return f"Error calling WebdriverIO server: {e}"

@tool
def set_value_action(element_selector: str, value: str) -> str:
    """Set value for an element in Chrome browser using WebdriverIO server."""
    try:
        resp = requests.post(f"{SERVER_URL}/set-value", json={"selector": element_selector, "value": value})
        return resp.text
    except Exception as e:
        return f"Error calling WebdriverIO server: {e}"

@tool
def get_page_source() -> str:
    """Get the HTML source of the current page using WebdriverIO server."""
    try:
        resp = requests.post(f"{SERVER_URL}/source")
        return resp.text
    except Exception as e:
        return f"Error calling WebdriverIO server: {e}"

In [5]:
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType

tools = [open_browser, navigate_to_url, click_action, set_value_action, get_page_source]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

def scenario_test_tool(scenario: str):
    """
    Accepts a scenario as input and performs testing using the defined tools and an LLM agent.
    After testing, generates a WebdriverIO (JavaScript) test script for the scenario.
    """
    prompt_template = PromptTemplate.from_template(
        """
        Act as QA Tets Automation Enginner, 
        perform the necessary web testing step by step:\n{scenario}
        Utilize all available tools to interact with the web application.
        Ensure to follow the scenario instructions carefully and report any issues found.
        """
    )
    prompt = prompt_template.format(scenario=scenario)
    response = agent.run(prompt)
    
    # Generate WebdriverIO test code after testing
    webdriverio_test_prompt = (
        f"Generate a WebdriverIO (JavaScript) test script for the following scenario:\n{scenario}"
    )
    wdio_test_code = llm.invoke(webdriverio_test_prompt)
    print(wdio_test_code)
    js_code = wdio_test_code.content
    with open("./webdriverio_generated_tests/saucedemo_test.js", "w", encoding="utf-8") as f:
        f.write(js_code)
    return response

scenario = (
    "Scenario:\n"
    "1. Navigate to https://www.saucedemo.com/v1/index.html.\n"
    "2. Login with username standard_user and password secret_sauce.\n"
    "3. Add the first 2 products to the cart.\n"
    "4. Click the cart icon and go to the cart page.\n"
    "5. Verify 2 items are added and visible in the cart."
)

result = scenario_test_tool(scenario)
print(result)


C:\Users\srini\AppData\Local\Temp\ipykernel_1740\794879369.py:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
C:\Users\srini\AppData\Local\Temp\ipykernel_1740\794879369.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.



> Entering new AgentExecutor chain...

Invoking: `open_browser` with `{}`


Browser opened
Invoking: `navigate_to_url` with `{'url': 'https://www.saucedemo.com/v1/index.html'}`


Navigated to https://www.saucedemo.com/v1/index.html
Invoking: `set_value_action` with `{'element_selector': '#user-name', 'value': 'standard_user'}`


Set value for #user-name
Invoking: `set_value_action` with `{'element_selector': '#password', 'value': 'secret_sauce'}`


Set value for #password
Invoking: `click_action` with `{'element_selector': '#login-button'}`


Clicked #login-button
Invoking: `click_action` with `{'element_selector': '#inventory_container .inventory_list .inventory_item:nth-child(1) .btn_inventory'}`


Clicked #inventory_container .inventory_list .inventory_item:nth-child(1) .btn_inventory
Invoking: `click_action` with `{'element_selector': '#inventory_container .inventory_list .inventory_item:nth-child(2) .btn_inventory'}`


Clicked #inventory_container .inventory_list .inventory_item